# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'buy'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = ['Trend']

work_timeframe = '1h'
higher_timeframe = '4h'
opt_limit = 10000
load = False

print(f'Timeframe is {work_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'vol_window': [96],
                            'vol_q_high': [0.75], # np.arange(0.1, 1, 0.05), 
                            'vol_q_low': [0.1], # np.arange(0.1, 1, 0.05),
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.5], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.5], # np.arange(0.5, 1, 0.05)
                            },
                'Trend': {
                            'timeperiod': [30], # np.arange(2, 50, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load, 'optimize')

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 24


100%|██████████| 24/24 [31:11<00:00, 77.98s/it]  


# Check local statistics

In [3]:
# result
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
17,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,36,0,2696,1.335208,2.377917,903.721667,6410.863333,1.2417,1.2038,1.2219,1.2146,1.2443,1.2599,1.2911,1.3169,1.3064,1.3109,1.3110,1.3274,1.3436,1.3710,1.4024,1.4203,1.4110,1.4168,1.4069,1.4008,1.4041,1.4120,1.4059,1.4003,0.25,0.49,0.73,0.96,1.19,1.41,1.62,1.83,2.03,2.22,2.40,2.58,2.74,2.88,3.01,3.12,3.21,3.29,3.36,3.45,3.55,3.59,3.59,3.57
12,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,26,0,2807,1.337917,2.373333,948.532083,6661.946667,1.2534,1.2095,1.2314,1.2195,1.2429,1.2594,1.2914,1.3174,1.3015,1.3073,1.3124,1.3322,1.3479,1.3764,1.4045,1.4195,1.4110,1.4148,1.4062,1.4037,1.4097,1.4183,1.4140,1.4057,0.25,0.49,0.73,0.96,1.19,1.40,1.62,1.83,2.03,2.22,2.40,2.57,2.73,2.87,3.01,3.11,3.20,3.28,3.35,3.44,3.54,3.59,3.58,3.57
13,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,28,0,2785,1.347317,2.371667,967.276917,6605.091667,1.2581,1.2156,1.2368,1.2257,1.2516,1.2687,1.3004,1.3262,1.3116,1.3190,1.3243,1.3442,1.3602,1.3873,1.4166,1.4331,1.4231,1.4273,1.4182,1.4126,1.4171,1.4254,1.4204,1.4121,0.25,0.49,0.73,0.96,1.18,1.40,1.61,1.83,2.02,2.21,2.40,2.57,2.73,2.87,3.00,3.11,3.20,3.28,3.36,3.44,3.54,3.59,3.58,3.57
16,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,34,0,2708,1.338846,2.371667,917.594517,6422.473333,1.2457,1.2081,1.2264,1.2179,1.2470,1.2629,1.2953,1.3210,1.3112,1.3150,1.3157,1.3322,1.3482,1.3756,1.4066,1.4241,1.4149,1.4203,1.4099,1.4035,1.4063,1.4144,1.4078,1.4023,0.25,0.49,0.73,0.96,1.18,1.40,1.61,1.83,2.02,2.21,2.40,2.57,2.73,2.87,3.00,3.11,3.20,3.28,3.36,3.44,3.54,3.59,3.58,3.57
14,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,30,0,2766,1.349604,2.371250,967.005125,6558.877500,1.2536,1.2129,1.2358,1.2292,1.2569,1.2738,1.3053,1.3325,1.3188,1.3254,1.3262,1.3451,1.3612,1.3880,1.4188,1.4359,1.4259,1.4302,1.4203,1.4137,1.4191,1.4271,1.4210,1.4138,0.25,0.49,0.73,0.96,1.18,1.40,1.61,1.82,2.02,2.21,2.40,2.57,2.73,2.87,3.00,3.11,3.20,3.28,3.36,3.44,3.54,3.59,3.58,3.57
15,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,32,0,2737,1.335279,2.370833,917.659079,6488.970833,1.2441,1.2037,1.2231,1.2152,1.2429,1.2589,1.2912,1.3164,1.3053,1.3099,1.3111,1.3288,1.3454,1.3724,1.4035,1.4210,1.4113,1.4166,1.4066,1.4003,1.4027,1.4114,1.4058,1.3991,0.25,0.49,0.73,0.96,1.18,1.40,1.61,1.83,2.02,2.21,2.40,2.57,2.73,2.87,3.00,3.11,3.20,3.28,3.35,3.44,3.54,3.58,3.58,3.57
18,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,38,0,2673,1.327629,2.370000,875.752762,6335.010000,1.2430,1.1995,1.2140,1.2063,1.2369,1.2535,1.2860,1.3094,1.2980,1.3021,1.3028,1.3210,1.3373,1.3652,1.3952,1.4118,1.4012,1.4072,1.3963,1.3907,1.3946,1.4023,1.3968,1.3920,0.25,0.49,0.73,0.96,1.18,1.40,1.61,1.83,2.02,2.21,2.40,2.57,2.73,2.87,3.00,3.11,3.20,3.28,3.35,3.43,3.54,3.58,3.58,3.56
20,Pattern_Trend,96,0.75,0.1,1,5,0.5,0.5,42,0,2633,1.319771,2.370000,841.956604,6240.210000,1.2011,1.1775,1.1985,1.1925,1.2273,1.2452,1.2789,1.2990,1.2916,1.2959,1.2964,1.3152,1.3312,1.3597,1.3909,1.4084,1.3981,1.4046,1.3938,1.3885,1.3916,1.3997,1.3956,1.3933,0.25,0.49,0.73,0.96,1.18,1.40,1.61,1.82,2.02,2.21,2.40,2.57,2.73,2.87,3.00,3.11,3.20,3.28,3.35,3.44,3.54,3.58,3.58,3.56
21,Pattern_Trend,96,0.75,0.1,1,5,0

In [4]:
# vol_q
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
109,Pattern_Trend,48,0.40,0.15,1,5,0.5,0.5,24,0,51,2.756179,3.632083,89.565137,185.236250,2.0444,1.7701,1.7547,2.0131,1.9073,2.0792,2.0801,2.1171,2.3009,2.3860,2.8780,2.8177,2.7920,3.0067,3.0703,2.9843,3.0088,2.9948,3.4585,3.4615,3.5315,3.9248,3.9147,3.8518,0.29,0.56,0.91,1.20,1.54,1.87,2.22,2.58,2.97,3.32,3.68,4.01,4.32,4.34,4.51,4.71,4.91,5.07,5.35,5.55,5.55,5.83,5.89,5.99
127,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.5,24,0,48,2.723083,3.843750,82.708000,184.500000,1.9650,1.7320,1.6510,1.9308,1.8550,2.0110,2.0383,2.0775,2.2673,2.3447,2.8497,2.7890,2.7632,2.9848,3.0493,2.9635,2.9764,2.9687,3.4533,3.4563,3.5276,3.9281,3.9178,3.8537,0.33,0.62,0.97,1.31,1.67,2.02,2.39,2.77,3.18,3.56,3.94,4.28,4.61,4.63,4.81,5.01,5.21,5.37,5.59,5.79,5.77,6.06,6.13,6.23
145,Pattern_Trend,48,0.50,0.15,1,5,0.5,0.5,24,0,46,2.680521,3.907083,77.303958,179.725833,1.9241,1.6390,1.6104,1.8905,1.7849,1.9260,1.9627,2.0032,2.2035,2.2695,2.7931,2.7337,2.7162,2.9665,3.0517,2.9463,2.9597,2.9411,3.4417,3.4453,3.5147,3.9279,3.8772,3.8036,0.35,0.65,1.02,1.36,1.72,2.06,2.43,2.86,3.23,3.62,4.00,4.35,4.68,4.70,4.88,5.08,5.28,5.44,5.67,5.87,5.85,6.15,6.21,6.31
91,Pattern_Trend,48,0.35,0.15,1,5,0.5,0.5,24,0,53,2.642729,3.385833,87.064646,179.449167,2.0098,1.8400,1.8110,2.0567,1.9492,2.0968,2.0957,2.1315,2.2900,2.2622,2.7215,2.6204,2.6030,2.8175,2.8839,2.8100,2.8347,2.8294,3.2680,3.2714,3.3137,3.6728,3.6517,3.5846,0.27,0.54,0.86,1.14,1.48,1.79,2.12,2.46,2.82,3.14,3.47,3.77,4.06,4.07,4.21,4.38,4.57,4.71,4.96,5.14,5.11,5.35,5.39,5.45
110,Pattern_Trend,48,0.40,0.20,1,5,0.5,0.5,24,0,53,2.755683,3.369167,93.051217,178.565833,2.1294,1.8261,1.8460,2.0728,1.9613,2.1250,2.1209,2.1558,2.2957,2.3778,2.8427,2.7858,2.7621,2.9723,3.0353,2.9539,2.9780,2.9676,3.4192,3.4331,3.5017,3.8845,3.8749,3.8145,0.26,0.51,0.82,1.08,1.40,1.70,2.02,2.36,2.71,3.04,3.38,3.70,4.00,4.01,4.16,4.34,4.55,4.72,4.99,5.20,5.21,5.49,5.56,5.65
128,Pattern_Trend,48,0.45,0.20,1,5,0.5,0.5,24,0,50,2.723746,3.556250,86.187292,177.812500,2.0538,1.7910,1.7486,1.9949,1.9118,2.0594,2.0811,2.1182,2.2632,2.3378,2.8147,2.7575,2.7337,2.9505,3.0144,2.9332,2.9462,2.9418,3.4134,3.4275,3.4974,3.8869,3.8772,3.8157,0.30,0.56,0.88,1.18,1.51,1.83,2.18,2.53,2.90,3.26,3.61,3.94,4.25,4.26,4.42,4.61,4.81,4.97,5.20,5.41,5.41,5.69,5.77,5.87
163,Pattern_Trend,48,0.55,0.15,1,5,0.5,0.5,24,0,43,2.861875,4.088750,80.060625,175.816250,2.0779,1.7630,1.7530,2.0450,1.9333,2.0955,2.1160,2.1494,2.3520,2.4180,2.9832,2.9116,2.8865,3.1385,3.2213,3.1025,3.1385,3.1090,3.6550,3.6600,3.7666,4.2121,4.1419,4.0552,0.35,0.68,1.07,1.42,1.80,2.17,2.57,2.97,3.37,3.77,4.16,4.53,4.88,4.90,5.10,5.31,5.53,5.70,5.93,6.15,6.13,6.46,6.53,6.65
135,Pattern_Trend,48,0.45,0.55,1,5,0.5,0.5,24,0,77,2.462362,2.260833,112.601912,174.084167,2.0900,1.9030,1.7374,1.9251,1.8365,1.9362,1.9540,2.0018,2.1168,2.1701,2.5318,2.4870,2.4352,2.6039,2.6489,2.5931,2.6103,2.6059,2.9633,2.9588,3.0288,3.3165,3.3243,3.3180,0.15,0.28,0.48,0.66,0.86,1.06,1.29,1.54,1.80,2.07,2.32,2.55,2.77,2.78,2.88,3.02,3.15,3.24,3.40,3.50,3.47,3.62,3.67,3.70
146,Pattern_Trend,48,0.50,0.20,1,5,0.5,0.5,24,0,48

In [3]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,48,0.45,0.15,1,5,0.50,0.60,24,0,43,3.000133,4.312917,86.005733,185.455417,2.3420,1.9716,1.8341,2.1379,2.0615,2.2398,2.2437,2.2753,2.4856,2.5649,3.1239,3.1122,3.0618,3.2932,3.3492,3.2415,3.2771,3.2463,3.7776,3.7795,3.8524,4.2951,4.2710,4.1660,0.35,0.69,1.08,1.44,1.85,2.25,2.66,3.09,3.55,3.97,4.39,4.78,5.15,5.19,5.41,5.64,5.87,6.05,6.31,6.55,6.56,6.79,6.87,7.02
3,Pattern_Trend,48,0.45,0.15,1,5,0.50,0.65,24,0,43,3.000133,4.312917,86.005733,185.455417,2.3420,1.9716,1.8341,2.1379,2.0615,2.2398,2.2437,2.2753,2.4856,2.5649,3.1239,3.1122,3.0618,3.2932,3.3492,3.2415,3.2771,3.2463,3.7776,3.7795,3.8524,4.2951,4.2710,4.1660,0.35,0.69,1.08,1.44,1.85,2.25,2.66,3.09,3.55,3.97,4.39,4.78,5.15,5.19,5.41,5.64,5.87,6.05,6.31,6.55,6.56,6.79,6.87,7.02
1,Pattern_Trend,48,0.45,0.15,1,5,0.50,0.55,24,0,44,2.898167,4.204167,83.519333,184.983333,2.2026,1.9103,1.7493,2.0445,1.9852,2.1478,2.1619,2.1947,2.3984,2.4773,3.0160,3.0067,2.9616,3.1949,3.2538,3.1531,3.1636,3.1396,3.6551,3.6575,3.7301,4.1582,4.1460,4.0478,0.36,0.68,1.06,1.42,1.81,2.20,2.60,3.02,3.47,3.89,4.29,4.67,5.02,5.05,5.27,5.49,5.71,5.88,6.14,6.37,6.38,6.60,6.69,6.83
0,Pattern_Trend,48,0.45,0.15,1,5,0.50,0.50,24,0,48,2.723083,3.843750,82.708000,184.500000,1.9650,1.7320,1.6510,1.9308,1.8550,2.0110,2.0383,2.0775,2.2673,2.3447,2.8497,2.7890,2.7632,2.9848,3.0493,2.9635,2.9764,2.9687,3.4533,3.4563,3.5276,3.9281,3.9178,3.8537,0.33,0.62,0.97,1.31,1.67,2.02,2.39,2.77,3.18,3.56,3.94,4.28,4.61,4.63,4.81,5.01,5.21,5.37,5.59,5.79,5.77,6.06,6.13,6.23
13,Pattern_Trend,48,0.45,0.15,1,5,0.55,0.65,24,0,41,3.091221,4.368750,85.740054,179.118750,2.1356,1.8355,1.7716,2.0939,1.9995,2.2206,2.3948,2.4238,2.6475,2.7182,3.2892,3.3027,3.2329,3.4404,3.4996,3.3771,3.4151,3.3748,3.9319,3.9181,3.9875,4.4455,4.4008,4.3327,0.35,0.68,1.07,1.44,1.86,2.27,2.70,3.13,3.61,4.03,4.45,4.84,5.21,5.25,5.48,5.71,5.95,6.14,6.41,6.65,6.65,6.88,6.97,7.12
12,Pattern_Trend,48,0.45,0.15,1,5,0.55,0.60,24,0,41,3.091221,4.368750,85.740054,179.118750,2.1356,1.8355,1.7716,2.0939,1.9995,2.2206,2.3948,2.4238,2.6475,2.7182,3.2892,3.3027,3.2329,3.4404,3.4996,3.3771,3.4151,3.3748,3.9319,3.9181,3.9875,4.4455,4.4008,4.3327,0.35,0.68,1.07,1.44,1.86,2.27,2.70,3.13,3.61,4.03,4.45,4.84,5.21,5.25,5.48,5.71,5.95,6.14,6.41,6.65,6.65,6.88,6.97,7.12
11,Pattern_Trend,48,0.45,0.15,1,5,0.55,0.55,24,0,42,2.980633,4.254583,83.186600,178.692500,2.0131,1.7818,1.6888,2.0010,1.9258,2.1274,2.2976,2.3286,2.5449,2.6161,3.1662,3.1804,3.1177,3.3299,3.3925,3.2785,3.2888,3.2567,3.7966,3.7844,3.8541,4.2965,4.2656,4.2022,0.36,0.67,1.06,1.42,1.82,2.22,2.64,3.06,3.52,3.94,4.34,4.72,5.08,5.11,5.33,5.55,5.78,5.97,6.23,6.46,6.46,6.69,6.77,6.91
10,Pattern_Trend,48,0.45,0.15,1,5,0.55,0.50,24,0,46,2.791067,3.871667,82.389067,178.096667,1.8009,1.6180,1.5937,1.8884,1.8000,1.9894,2.1519,2.1907,2.3915,2.4624,2.9777,2.9307,2.8906,3.0949,3.1640,3.0678,3.0808,3.0676,3.5738,3.5642,3.6332,4.0461,4.0196,3.9877,0.32,0.61,0.96,1.30,1.67,2.03,2.41,2.80,3.21,3.60,3.97,4.32,4.64,4.66,4.84,5.05,5.25,5.42,5.64,5.84,5.81,6.11,6.18,6.28
31,Pattern_Trend,48,0.45,0.15,1,5,0.65,0.55,24

In [2]:
# window
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,24,0.45,0.15,1,5,0.5,0.6,24,0,38,2.929742,4.686250,73.330183,178.077500,2.3852,2.1633,2.1428,2.4822,2.3952,2.3997,2.2941,2.2215,2.4078,2.3692,2.5052,2.4399,2.4134,2.9995,3.0765,2.9785,3.0549,3.0746,3.7418,3.7557,3.8905,4.4404,4.4100,4.2719,0.37,0.75,1.18,1.56,2.05,2.48,2.93,3.36,3.86,4.24,4.65,4.99,5.33,5.49,5.73,6.01,6.30,6.55,6.86,7.18,7.27,7.60,7.75,7.98
3,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,24,0,42,3.087479,4.496250,87.674125,188.842500,2.4045,2.0149,1.8693,2.1894,2.1074,2.2855,2.3216,2.3515,2.5659,2.6448,3.2181,3.2043,3.1493,3.3774,3.4312,3.3369,3.3725,3.3358,3.8904,3.8995,3.9775,4.4383,4.4128,4.3007,0.37,0.71,1.11,1.47,1.90,2.30,2.72,3.16,3.63,4.06,4.49,4.90,5.28,5.34,5.59,5.85,6.11,6.32,6.61,6.88,6.95,7.23,7.36,7.57
5,Pattern_Trend,96,0.45,0.15,1,5,0.5,0.6,24,0,38,3.474521,4.058750,94.031792,154.232500,2.6012,2.3784,2.2675,2.7234,2.7614,2.8815,2.6955,2.7399,2.8839,2.7604,3.3143,3.2293,3.1655,3.6946,3.7725,3.7604,3.7952,3.7036,4.2942,4.2786,4.5485,5.1363,5.0534,4.9490,0.30,0.58,0.91,1.28,1.70,2.11,2.54,2.98,3.40,3.81,4.18,4.54,4.89,4.90,5.10,5.32,5.53,5.68,5.90,6.10,6.09,6.38,6.50,6.69
4,Pattern_Trend,72,0.45,0.15,1,5,0.5,0.6,24,0,45,2.922613,3.856667,86.517562,173.550000,2.4178,2.1488,2.0846,2.3571,2.2482,2.3329,2.2769,2.2899,2.4544,2.4082,2.9282,2.8711,2.8328,3.0687,3.1310,3.0532,3.0863,3.0620,3.5722,3.5812,3.6934,4.1270,4.1057,4.0111,0.33,0.64,0.99,1.31,1.68,2.02,2.38,2.80,3.17,3.54,3.90,4.25,4.57,4.58,4.76,4.97,5.19,5.36,5.62,5.85,5.89,6.15,6.23,6.38
1,Pattern_Trend,12,0.45,0.15,1,5,0.5,0.6,24,0,35,1.721800,3.044167,25.263000,106.545833,1.9447,1.9343,2.0239,1.8010,1.6517,1.5963,1.3794,1.3209,1.2744,1.2051,1.2561,1.2214,1.2059,1.1127,1.3123,1.2892,1.3676,1.4632,2.1314,2.1378,2.3068,2.8425,2.8120,2.7326,0.29,0.60,0.96,1.22,1.56,1.85,2.13,2.40,2.69,2.90,3.13,3.32,3.48,3.65,3.67,3.78,3.92,4.04,4.33,4.52,4.40,4.67,4.73,4.82
0,Pattern_Trend,6,0.45,0.15,1,5,0.5,0.6,24,0,24,2.933033,2.931250,46.392800,70.350000,2.5630,2.3165,2.4593,2.0829,1.9465,2.3162,2.2588,2.3620,2.2434,1.9607,2.8039,2.7689,2.7387,2.5773,2.7371,2.7295,2.8279,3.0100,3.8494,3.8359,3.9873,4.8123,4.6795,4.5258,0.27,0.53,0.82,1.16,1.44,1.76,2.06,2.44,2.72,3.01,3.25,3.47,3.68,3.83,3.76,3.79,3.89,3.95,4.10,4.23,3.79,4.07,4.09,4.24


In [2]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
3,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,8,0,30,3.359379,4.279167,70.781375,128.375000,2.5158,2.1701,2.0954,2.1656,2.0307,2.3161,2.1444,2.2455,2.5686,2.6765,3.4676,3.4367,3.3696,3.6669,3.8526,3.7060,3.7895,3.7771,4.3551,4.3334,4.5150,5.1770,5.1788,5.0711,0.30,0.63,0.97,1.30,1.69,2.10,2.45,2.90,3.36,3.78,4.19,4.62,5.03,4.97,5.21,5.49,5.80,6.10,6.43,6.74,6.69,7.07,7.27,7.61
4,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,10,0,30,3.236583,4.218750,67.097500,126.562500,2.4023,2.0363,2.0148,2.1004,1.9010,2.1847,2.0360,2.1365,2.4574,2.5679,3.3525,3.3113,3.2623,3.5871,3.7035,3.5675,3.6497,3.6457,4.2148,4.1961,4.3752,5.0245,5.0264,4.9241,0.30,0.63,0.97,1.30,1.68,2.08,2.43,2.88,3.33,3.74,4.15,4.57,4.97,4.91,5.14,5.40,5.71,6.00,6.32,6.62,6.56,6.95,7.14,7.47
5,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,12,0,31,3.118771,4.130000,65.681896,128.030000,2.3763,1.9646,1.9406,2.0426,1.8207,2.0981,1.9690,2.0655,2.3883,2.4969,3.2551,3.2578,3.2111,3.5220,3.6362,3.4935,3.5688,3.5308,4.0141,3.9943,4.1133,4.7257,4.7278,4.6374,0.29,0.61,0.94,1.27,1.65,2.05,2.40,2.85,3.30,3.71,4.12,4.53,4.93,4.87,5.08,5.33,5.61,5.88,6.17,6.44,6.36,6.71,6.87,7.15
6,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,14,0,32,3.119504,3.967083,67.824133,126.946667,2.4080,1.9457,1.9847,2.0990,1.8381,2.1335,1.9966,2.0720,2.3827,2.5091,3.2613,3.2640,3.2176,3.5248,3.6378,3.4963,3.5709,3.5330,4.0010,3.9815,4.0860,4.6696,4.6716,4.5833,0.29,0.58,0.91,1.23,1.59,1.96,2.31,2.73,3.17,3.58,3.98,4.39,4.78,4.71,4.91,5.13,5.40,5.65,5.92,6.18,6.08,6.40,6.55,6.78
7,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,16,0,38,3.350054,3.902917,89.302058,148.310833,2.3693,2.1370,1.9974,2.4157,2.2073,2.4691,2.4505,2.5174,2.8042,2.9036,3.5620,3.5608,3.4991,3.7113,3.8046,3.6696,3.7364,3.6987,4.1741,4.1650,4.2570,4.7902,4.7921,4.7089,0.22,0.46,0.74,1.03,1.38,1.77,2.15,2.65,3.11,3.53,3.93,4.33,4.71,4.69,4.90,5.13,5.39,5.63,5.92,6.16,6.11,6.41,6.55,6.77
8,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,18,0,40,3.201183,3.395417,88.047333,135.816667,2.4582,2.1304,1.9656,2.3721,2.1785,2.4058,2.3959,2.4615,2.7046,2.7775,3.4042,3.3740,3.3217,3.5277,3.5838,3.4660,3.5278,3.5030,3.9587,3.9502,4.0241,4.5040,4.4763,4.3568,0.20,0.40,0.64,0.90,1.22,1.56,1.91,2.36,2.76,3.14,3.50,3.85,4.18,4.12,4.27,4.47,4.68,4.88,5.11,5.32,5.23,5.48,5.58,5.73
9,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,20,0,43,3.063704,3.049167,88.739279,131.114167,2.3774,2.0335,1.8377,2.2301,2.0727,2.2808,2.2798,2.3452,2.5658,2.6482,3.2434,3.2081,3.1717,3.3579,3.4191,3.3187,3.3732,3.3572,3.8154,3.8173,3.8916,4.3480,4.3228,4.2133,0.18,0.36,0.57,0.80,1.09,1.40,1.71,2.12,2.48,2.83,3.16,3.48,3.78,3.71,3.83,4.01,4.20,4.37,4.59,4.78,4.68,4.91,5.01,5.13
10,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,22,0,44,3.047437,3.024583,90.087250,133.081667,2.3886,2.0765,1.8661,2.2522,2.0821,2.2537,2.2559,2.3205,2.5375,2.6194,3.2088,3.1749,3.1486,3.3345,3.3960,3.2979,3.3517,3.3370,3.7900,3.7919,3.8659,4.3167,4.2920,4.1801,0.18,0.36,0.57,0.80,1.08,1.39,1.70,2.10,2.46,2.80,3.12,3.44,3.74,3.68,3.80,3.98,4.17,4.34,4.56,4.75,4.65,4.87,4.97,5.08
11,Pattern_Trend,48,0.45,0.15,1,5,0.5,0.6,24,0,46,3.053750,2.88958

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [14]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'Trend']
work_timeframe = '1h'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {
                            'vol_window': [48],
                            'vol_q_high': [0.4],
                            'vol_q_low': [0.1],
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.6],
                            'second_candle': [0.55],
                            },
                'Trend': {
                            'timeperiod': [48],
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)